<a href="https://colab.research.google.com/github/WRFitch/fyp/blob/main/src/fyp_model_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A notebook for testing an exported model 

In [ ]:
# Import fyputil library
!git clone https://github.com/WRFitch/fyp.git
%cd /content/fyp/src/fyputil
import constants as const
import ee_utils as eeutil
import fyp_utils as fyputil
%cd /content